In [3]:
import os, sys

# tambah path root supaya 'src' bisa di-import
sys.path.append(os.path.abspath(".."))

from src.config import GOOGLE_API_KEY

if GOOGLE_API_KEY:
    print("API key terbaca, panjang:", len(GOOGLE_API_KEY))
else:
    print("API key KOSONG – cek file .env dan config.py")


API key terbaca, panjang: 39


In [4]:
from src.llm_features import get_gemini_model

model = get_gemini_model()  # default "gemini-2.5-flash"
print("Model siap dipakai:", type(model))


Model siap dipakai: <class 'google.generativeai.generative_models.GenerativeModel'>


In [6]:
from src.llm_features import get_gemini_model

model = get_gemini_model("gemini-2.5-flash")

PROMPT_INSTRUKSI = """
Kamu adalah analis konten media sosial Indonesia yang bertugas menilai
seberapa kuat emosi, seberapa sensasional format, dan seberapa rendah
kredibilitas sumber sebuah postingan.

⚠ PENTING:
- Kamu TIDAK diminta memutuskan "hoaks atau bukan".
- Kamu HANYA memberi tiga skor dasar (0–1) yang nanti akan digunakan
  sistem lain untuk menghitung:
  - potensi_viral (seberapa mudah menyebar), dan
  - resiko_hoaks (seberapa berisiko mengandung hoaks).
- Jadi fokusmu adalah menilai tiga faktor dasar ini seobjektif mungkin.

==================================================
DEFINISI TIGA PARAMETER
==================================================

1) intensitas_emosi (0–1)
   Mengukur seberapa emosional isi postingan.

   0.0  ≈ sangat netral / informatif
   0.3  ≈ agak menyentuh, tapi masih tenang
   0.5  ≈ cukup emosional (sedih, khawatir, iba)
   0.8  ≈ sangat emosional (takut, panik, marah)
   1.0  ≈ sangat provokatif / menakut-nakuti / penuh kemarahan

   Contoh kalimat yang menaikkan intensitas_emosi:
   - banyak kata "musibah", "bencana besar", "azab", "bahaya", "ancaman",
     "korban berjatuhan", dsb.
   - nada panik, menyalahkan pihak tertentu, atau menghakimi.

2) kecurigaan_format (0–1)
   Mengukur seberapa sensasional / mirip "pesan berantai" format postingan.

   0.0  ≈ format sangat wajar: huruf biasa, tanda baca normal,
          tidak ada ajakan "sebar ke semua orang".
   0.3  ≈ ada sedikit dramatisasi (1–2 kata kapital, 1 tanda seru),
          tapi masih terasa seperti pengumuman normal.
   0.5  ≈ cukup sensasional: beberapa kata kapital, gaya bahasa hiperbolik.
   0.8  ≈ sangat sensasional: BANYAK HURUF KAPITAL, BANYAK !!!,
          judul heboh, gaya clickbait.
   1.0  ≈ pola klasik "forward pesan berantai" / "broadcast hoaks":
          penuh CAPS, !!!, dan ajakan menyebarkan.

   Hal-hal yang MENAIKKAN kecurigaan_format:
   - BANYAK huruf kapital (SELURUH KALIMAT DALAM CAPS).
   - Banyak tanda seru (!!!), tanda tanya (???), atau kombinasi aneh.
   - Kalimat seperti: "SEGERA SEBARKAN", "FORWARD KE SEMUA KONTAK",
     "JANGAN ABAIKAN PESAN INI", "VIRALKAN SEKARANG".

   Hal-hal yang MENURUNKAN kecurigaan_format:
   - Desain poster lembaga resmi dengan logo jelas, informasi rapi,
     gaya bahasa formal, dan TANPA ajakan menyebarkan pesan berantai.

3) kredibilitas_rendah (0–1)
   Mengukur seberapa TIDAK kredibel sumber / akun yang mengunggah.

   0.0  ≈ sangat kredibel:
          - lembaga pemerintah, media arus utama, NGO resmi,
          - tokoh publik yang dikenal luas,
          - informasi kontak dan identitas jelas.
   0.3  ≈ cenderung kredibel tapi tidak resmi (komunitas besar, organisasi yang dikenal).
   0.5  ≈ netral / meragukan (akun pribadi biasa, identitas tidak terlalu jelas).
   0.8  ≈ sangat meragukan (nama akun aneh, bio tidak jelas, tampak seperti akun spam).
   1.0  ≈ sangat tidak kredibel (akun baru, tidak jelas pemiliknya, konten penuh provokasi).

   Catatan penting:
   - Jika postingan berasal dari lembaga resmi atau tokoh publik terkenal
     (misalnya kementerian, lembaga penanggulangan bencana, media nasional,
      organisasi kemanusiaan resmi), maka kredibilitas_rendah seharusnya kecil
     (misal 0.0–0.3), kecuali ada indikasi kuat sebaliknya.

==================================================
KETERKAITAN DENGAN VIRAL & HOAKS
==================================================

Sistem lain akan menggunakan:
- intensitas_emosi + kecurigaan_format → untuk menghitung potensi_viral
- kecurigaan_format + kredibilitas_rendah → untuk menghitung resiko_hoaks

Tugasmu BUKAN menghitung viral/hoaks, tetapi memberikan nilai dasar yang konsisten:
- Konten bisa saja sangat viral tetapi resiko hoaks rendah
  (contoh: penggalangan dana resmi, kampanye kemanusiaan).
- Konten bisa saja tidak terlalu viral tetapi resiko hoaks tinggi
  (contoh: klaim medis palsu dari akun kecil).

==================================================
FORMAT JAWABAN
==================================================

Jawab SELALU dalam format JSON murni seperti ini:

{
  "intensitas_emosi": 0.8,
  "kecurigaan_format": 0.9,
  "kredibilitas_rendah": 0.7
}

- Gunakan titik (.) sebagai pemisah desimal.
- Jangan menambahkan teks lain di luar JSON.
- Jika ragu, gunakan nilai tengah (misal 0.3, 0.5, 0.7) daripada 0.0 atau 1.0 ekstrem.

==================================================
CONTOH PENILAIAN
==================================================

[Contoh 1 – Donasi resmi, emosional tapi tidak mencurigakan]
...

[Contoh 2 – Pesan berantai hoaks]
...

[Contoh 3 – Berita serius dari media resmi]
...

[Contoh 4 – Rumor dari akun pribadi meragukan]
...
"""

teks_uji = "SEGERA SEBARKAN!!! VIRUS BARU MENYERANG!! MINUM AIR GARAM BIAR SELAMAT!!! KIRIM KE SEMUA TEMAN ANDA!!!"

prompt = PROMPT_INSTRUKSI + "\n\n==================================================\nTEKS YANG HARUS DINILAI\n==================================================\n" + teks_uji

resp = model.generate_content(prompt)
print(resp.text)


```json
{
  "intensitas_emosi": 0.9,
  "kecurigaan_format": 1.0,
  "kredibilitas_rendah": 0.9
}
```


In [13]:
import os, sys
sys.path.append(os.path.abspath(".."))

import importlib
import src.llm_features as lf

lf = importlib.reload(lf)  # paksa pakai versi terbaru
print("File modul:", lf.__file__)
print("Nama-nama yang ada di llm_features:")
print([name for name in dir(lf) if not name.startswith("_")])


File modul: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\FP_KK\fp_kk_11_early_warning_hoax\src\llm_features.py
Nama-nama yang ada di llm_features:
['Any', 'Dict', 'GOOGLE_API_KEY', 'PROMPT_INSTRUKSI', 'annotations', 'genai', 'get_gemini_model', 'hitung_skor_llm_df', 'json', 'pd', 'skor_hoax_dari_teks']


In [14]:
from src.llm_features import get_gemini_model, skor_hoax_dari_teks, hitung_skor_llm_df

model = get_gemini_model("gemini-2.5-flash")

teks_uji = "SEGERA SEBARKAN!!! VIRUS BARU MENYERANG!! MINUM AIR GARAM BIAR SELAMAT!!! KIRIM KE SEMUA TEMAN ANDA!!!"

scores = skor_hoax_dari_teks(teks_uji, model=model)
scores


{'intensitas_emosi': 1.0, 'kecurigaan_format': 1.0, 'kredibilitas_rendah': 1.0}

In [15]:
import pandas as pd
from src.llm_features import hitung_skor_llm_df

df_test = pd.DataFrame({
    "text": [
        "SEGERA SEBARKAN!!! VIRUS BARU MENYERANG!! MINUM AIR GARAM BIAR SELAMAT!!!",
        "Pemerintah mengumumkan jadwal vaksinasi resmi di website Kementerian Kesehatan.",
    ]
})

df_hasil = hitung_skor_llm_df(df_test, text_col="text")
df_hasil


,text,intensitas_emosi,kecurigaan_format,kredibilitas_rendah
0,SEGERA SEBARKAN!!! VIRUS BARU MENYERANG!! MINU...,1.0,1.0,1.0
1,Pemerintah mengumumkan jadwal vaksinasi resmi ...,0.0,0.0,0.0


In [ ]:
from src.fuzzy_system import hitung_resiko_hoaks

# contoh 1: jelas hoaks-berantai (semua tinggi)
print(hitung_resiko_hoaks(0.9, 0.9, 0.9))

# contoh 2: info resmi (emosi sedang, format rapi, sumber kredibel)
print(hitung_resiko_hoaks(0.6, 0.2, 0.1))

# contoh 3: sumber meragukan tapi format netral
print(hitung_resiko_hoaks(0.3, 0.2, 0.8))


64.15912625477168
15.425619834710746
50.000000000000014


: 